# BigQuery - BQML - Export and Serving

In [22]:
%%bigquery
EXPORT MODEL `statmike-mlops.digits.digits_lr`
OPTIONS(URI = 'gs://statmike-models/digits/bqml')

""


In [23]:
MODEL_NAME='DIGIT_MODEL'
MODEL_DIR='gs://statmike-models/digits/bqml'
MODEL_VERSION='BQML_LR'
FRAMEWORK='TENSORFLOW'

This is a custom model.  We are importing it to AI Platform (Unified).  This is a good starting point:
- https://cloud.google.com/ai-platform-unified/docs/predictions/importing-custom-trained-model

Import the Model:
- https://cloud.google.com/ai-platform-unified/docs/predictions/importing-custom-trained-model#import_a_model_programmatically
- region of bucket and model must be the same

In [24]:
%%bash
gcloud beta ai models upload --region=us-central1 --display-name=BQML-Digits --container-image-uri='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest' --artifact-uri='gs://statmike-models/digits/bqml'

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8829706192180740096]...
.......................................................................................................................................done.


Deploy the Model:
    - https://cloud.google.com/ai-platform-unified/docs/predictions/deploy-model-api

In [25]:
%%bash
gcloud beta ai endpoints create --region=us-central1 --display-name=digits

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [1040730636643467264]...
........................................................................................................................................done.
Created AI Platform endpoint: projects/691911073727/locations/us-central1/endpoints/7050965758787977216.


In [28]:
!gcloud beta ai endpoints list --region=us-central1 --filter=display_name=digits

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
7050965758787977216  digits


In [31]:
!gcloud beta ai models list --region=us-central1 --filter=display_name=BQML-Digits

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
artifactUri: gs://statmike-models/digits/bqml
containerSpec:
  imageUri: us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest
createTime: '2021-04-01T23:56:52.847600Z'
displayName: BQML-Digits
etag: AMEw9yNzx8Zf-6jeftzU7Eo8hdHZCysFBCV5GI6Js-Qtur16mzyVsTvj1qLx6-l9ZfA=
name: projects/691911073727/locations/us-central1/models/5751325422570700800
predictSchemata: {}
supportedDeploymentResourcesTypes:
- DEDICATED_RESOURCES
supportedExportFormats:
- exportableContents:
  - ARTIFACT
  id: custom-trained
supportedInputStorageFormats:
- jsonl
- csv
- tf-record
- tf-record-gzip
- file-list
supportedOutputStorageFormats:
- jsonl
updateTime: '2021-04-01T23:57:16.157314Z'


In [32]:
!gcloud beta ai endpoints deploy-model 7050965758787977216 --region=us-central1 --model=5751325422570700800 --display-name=BQML-Digits

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [6776064762099793920]...done.                            
Deployed a model to the endpoint 7050965758787977216. Id of the deployed model: 9105310876310700032.


Get an online prediction:
- https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#online_predict_custom_trained-python

In [46]:
from typing import Dict

from google.cloud.aiplatform import gapic
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

ModuleNotFoundError: No module named 'google.cloud.aiplatform'

In [33]:
def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instance_dict: Dict,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

ImportError: cannot import name 'aiplatform' from 'google.cloud' (unknown location)

In [ ]:
predict_custom_trained_model_sample(
    project="691911073727",
    endpoint_id="7050965758787977216",
    location="us-central1",
    instance_dict={ "values": newob, "key": 1}
)

In [35]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`)
  )

In [43]:
newob = pred.loc[:0,'p0':'p63'].values.flatten().tolist()

[0.0,
 0.0,
 0.0,
 9.0,
 14.0,
 6.0,
 0.0,
 0.0,
 0.0,
 0.0,
 10.0,
 13.0,
 4.0,
 13.0,
 2.0,
 0.0,
 0.0,
 2.0,
 14.0,
 0.0,
 0.0,
 10.0,
 6.0,
 0.0,
 0.0,
 4.0,
 9.0,
 0.0,
 0.0,
 6.0,
 8.0,
 0.0,
 0.0,
 5.0,
 8.0,
 0.0,
 0.0,
 8.0,
 7.0,
 0.0,
 0.0,
 2.0,
 11.0,
 1.0,
 0.0,
 9.0,
 5.0,
 0.0,
 0.0,
 0.0,
 6.0,
 11.0,
 4.0,
 13.0,
 3.0,
 0.0,
 0.0,
 0.0,
 1.0,
 11.0,
 16.0,
 12.0,
 0.0,
 0.0]